In [ ]:
#!uv pip install -q nltk gensim pyLDAvis unidecode matplotlib seaborn pandas pyarrow

In [ ]:
from gensim.models import CoherenceModel, LdaModel
from gensim.utils import simple_preprocess
from pathlib import Path
import gensim
import gensim.corpora as corpora
import json
import matplotlib.pyplot  as plt
import numpy as np
import os
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim
import pyLDAvis.gensim_models as gensimvis
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
stop_words = set(stopwords.words('french'))

# Utils LDA

In [ ]:
def lda_model(processed_texts, num_topics=5, passes=10):
    os.environ["TOKENIZERS_PARALLELISM"] = "false"
    warnings.filterwarnings('ignore')
    dictionary = corpora.Dictionary(processed_texts)
    corpus = [dictionary.doc2bow(text) for text in processed_texts]
    model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=42, passes=passes)
    for topic in model.print_topics(num_words=5):
        print(topic)
    return model, corpus, dictionary
    
def visualize_lda(model, corpus, dictionary):
    pyLDAvis.enable_notebook()
    vis_data = gensimvis.prepare(model, corpus, dictionary)
    return pyLDAvis.display(vis_data)

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, max_topics=10):
    coherence_scores = []
    for num_topics in range(2, max_topics + 1):
        lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, passes=10, random_state=42)
        coherence_model = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_score = coherence_model.get_coherence()
        coherence_scores.append((num_topics, coherence_score))
        print(f"Num Topics: {num_topics}, Coherence Score: {coherence_score:.4f}")
    
    return coherence_scores

# Pour HS

In [ ]:
df_hs = pd.read_parquet("processed_texts_hs.parquet")
processed_texts_hs = df_hs["processed_texts_hs"].tolist()

In [ ]:
model_hs, corpus_hs, dictionary_hs = lda_model(processed_texts_hs)

In [ ]:
visualize_lda(model_hs, corpus_hs, dictionary_hs)

In [ ]:
#compute_coherence_values(dictionary_hs, corpus_hs, processed_texts_hs, max_topics=20)

In [ ]:
model_hs, corpus_hs, dictionary_hs = lda_model(processed_texts_hs, num_topics= 18)

In [ ]:
visualize_lda(model_hs, corpus_hs, dictionary_hs)

# Pour les données de santé

In [ ]:
df_sante = pd.read_parquet("processed_texts_sante.parquet")
processed_texts_sante = df_hs["processed_texts_sante"].tolist()

In [ ]:
model_sante, corpus_sante, dictionary_sante = lda_model(processed_texts_sante)

In [ ]:
visualize_lda(model_sante, corpus_sante, dictionary_sante)

In [ ]:
#compute_coherence_values(dictionary_sante, corpus_sante, processed_texts_sante, max_topics=20)

In [ ]:
model_sante, corpus_sante, dictionary_sante = lda_model(processed_texts_sante, num_topics= 17)

In [ ]:
visualize_lda(model_sante, corpus_sante, dictionary_sante)